In [2]:
import pandas as pd
import geopandas as gpd

# ETL

In [5]:
df=pd.read_excel('/Users/ruhidmirzayev/Palette/Data/Cohort5/Manitoba_yield_02_22.xlsx')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61044 entries, 0 to 61043
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Year                  61044 non-null  int64 
 1   Risk Area / R.M.      61044 non-null  object
 2   Crop                  61044 non-null  object
 3   Soil                  61044 non-null  object
 4   Farms                 61044 non-null  object
 5   Acres                 61044 non-null  object
 6   Yield/acre(Metric)    61044 non-null  object
 7   Yield/acre(Metric).1  24887 non-null  object
 8   Yield/acre(Imperial)  24887 non-null  object
dtypes: int64(1), object(8)
memory usage: 4.2+ MB


In [7]:
gdf=gpd.read_file('/Users/ruhidmirzayev/Palette/Data/Cohort5/MUNICIPALITY/MG_MUNICIPALITY_POLY.shp')

In [9]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    183 non-null    int64   
 1   MUNI_NO     183 non-null    int64   
 2   MUNI_NAME   183 non-null    object  
 3   MUNI_TYPE   183 non-null    object  
 4   MUNI_LIST_  183 non-null    object  
 5   MUNI_LIS_1  183 non-null    object  
 6   SHAPE_Leng  183 non-null    float64 
 7   SHAPE_Area  183 non-null    float64 
 8   geometry    183 non-null    geometry
dtypes: float64(2), geometry(1), int64(2), object(4)
memory usage: 13.0+ KB


## Data Standardizing

In [19]:
df_clean_1=df.rename(columns={'Risk Area / R.M.': 'RM', 'Yield/acre(Imperial)': 'Yield'})\
    .drop(columns=['Yield/acre(Metric)', 'Yield/acre(Metric).1'])

In [29]:
df_clean_2=df_clean_1.sort_values(['RM', 'Year', 'Crop'])

In [33]:
df_clean_2

,Year,RM,Crop,Soil,Farms,Acres,Yield
21394,2000,ALEXANDER,ARGENTINE CANOLA,F,12,"2,401.0",24.1 Bushels
21661,2000,ALEXANDER,ARGENTINE CANOLA,E,10,"1,509.0",23.3 Bushels
22851,2000,ALEXANDER,ARGENTINE CANOLA,G,10,930,19.0 Bushels
23794,2000,ALEXANDER,ARGENTINE CANOLA,H,5,"1,464.0",13.5 Bushels
24887,2000,ALEXANDER,ARGENTINE CANOLA,D,Below,Minimum,NaN
...,...,...,...,...,...,...,...
61041,2022,YELLOWHEAD,FLAX,B,Below,Minimum,NaN
61042,2022,YELLOWHEAD,FLAX,C,Below,Minimum,NaN
587,2022,YELLOWHEAD,OATS,C,10,"2,976.0",130.0 Bushels
629,2022,YELLOWHEAD,OATS,B,10,"1,889.0",128.3 Bushels


In [32]:
df_clean_2.pivot_table(index=['Year', 'RM', 'Soil', 'Farms','Acres'], columns='Crop',values='Yield' )

TypeError: agg function failed [how->mean,dtype->object]

In [ ]:
year - rm - 11 crop names - soil - farms- acres - yield

# EDA